In [4]:
import os
os.chdir('/home/damon/Github/mmlab/mmdetection/')
import os.path as osp
import sys
import numpy as np
import torch, torchvision
from torch import nn
from torch.utils import model_zoo
from torch.nn.parallel import DataParallel
from torch.utils.data import DataLoader
from torch.utils.data.dataset import ConcatDataset
# %load_ext autoreload
# %autoreload 2
from torchsummary import summary

In [5]:
import mmdet
import mmcv
from mmdet.models import build_detector
from mmcv.parallel import MMDataParallel
from mmdet.datasets import get_dataset
from mmdet.datasets import transforms

In [6]:
# Get cfg
import mmcv
config_file = 'visdrone/configs/ssd300.py'
cfg = mmcv.Config.fromfile(config_file)


# Get model
from mmdet.models import build_detector
ssd = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

missing keys in source state_dict: extra.0.weight, extra.2.bias, extra.7.bias, extra.3.weight, extra.1.weight, extra.1.bias, extra.5.bias, extra.6.weight, extra.2.weight, l2_norm.weight, extra.5.weight, extra.4.weight, extra.4.bias, extra.7.weight, extra.6.bias, extra.0.bias, extra.3.bias



loaded:  {'features.21.bias', 'features.33.weight', 'features.26.bias', 'features.21.weight', 'features.24.weight', 'features.14.weight', 'features.12.weight', 'features.28.bias', 'features.26.weight', 'features.10.weight', 'features.24.bias', 'features.12.bias', 'features.17.bias', 'features.0.bias', 'features.7.weight', 'features.17.weight', 'features.19.bias', 'features.31.bias', 'features.33.bias', 'features.2.bias', 'features.28.weight', 'features.2.weight', 'features.5.bias', 'features.19.weight', 'features.10.bias', 'features.7.bias', 'features.0.weight', 'features.5.weight', 'features.14.bias', 'features.31.weight'}


In [7]:
ssd = MMDataParallel(ssd)

# Dataset

In [8]:
from mmdet.datasets import get_dataset
train_ds = get_dataset(cfg.data.train)  # is a repeated dataset
train_ds = train_ds.dataset
# data_loader = mmdet.datasets.build_dataloader(train_ds, imgs_per_gpu=4, workers_per_gpu=1, dist=False)

extract_dc = lambda dc: {k:v.data for k, v in dc.items()}

data_loader = mmdet.datasets.build_dataloader(train_ds, imgs_per_gpu=4, workers_per_gpu=1, dist=False)
ssd_input = iter(data_loader).next()

loading annotations into memory...
Done (t=0.77s)
creating index...
index created!
load_annotations time: 0.9s from data/VisDrone2019-DET/VisDrone2018-DET-train/annotations_train.json


In [ ]:
ssd(**ssd_input)

# Backbone

In [9]:
input_size =  (3, 300, 300)
ssdmodule = ssd.module
backbone = ssd.module.backbone
bbox_head = ssd.module.bbox_head

In [ ]:
x = torch.zeros([1,3,300,300]).cuda()
fea_map = backbone(x)

In [13]:
summary(backbone.features, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 300, 300]           1,792
              ReLU-2         [-1, 64, 300, 300]               0
            Conv2d-3         [-1, 64, 300, 300]          36,928
              ReLU-4         [-1, 64, 300, 300]               0
         MaxPool2d-5         [-1, 64, 150, 150]               0
            Conv2d-6        [-1, 128, 150, 150]          73,856
              ReLU-7        [-1, 128, 150, 150]               0
            Conv2d-8        [-1, 128, 150, 150]         147,584
              ReLU-9        [-1, 128, 150, 150]               0
        MaxPool2d-10          [-1, 128, 75, 75]               0
           Conv2d-11          [-1, 256, 75, 75]         295,168
             ReLU-12          [-1, 256, 75, 75]               0
           Conv2d-13          [-1, 256, 75, 75]         590,080
             ReLU-14          [-1, 256,

In [12]:
summary(backbone, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 300, 300]           1,792
              ReLU-2         [-1, 64, 300, 300]               0
            Conv2d-3         [-1, 64, 300, 300]          36,928
              ReLU-4         [-1, 64, 300, 300]               0
         MaxPool2d-5         [-1, 64, 150, 150]               0
            Conv2d-6        [-1, 128, 150, 150]          73,856
              ReLU-7        [-1, 128, 150, 150]               0
            Conv2d-8        [-1, 128, 150, 150]         147,584
              ReLU-9        [-1, 128, 150, 150]               0
        MaxPool2d-10          [-1, 128, 75, 75]               0
           Conv2d-11          [-1, 256, 75, 75]         295,168
             ReLU-12          [-1, 256, 75, 75]               0
           Conv2d-13          [-1, 256, 75, 75]         590,080
             ReLU-14          [-1, 256,

In [ ]:
bbox_head

In [ ]:
reg_out = bbox_head(fea_map)[0]
[v.shape for v in reg_out]

In [ ]:
cls_out = bbox_head(fea_map)[1]
[v.shape for v in cls_out]

# bboxx_head loss

In [ ]:
lossinput = bbox_head(fea_map)

# Transfroms

In [ ]:
from mmdet.datasets import transforms

In [ ]:
img = np.zeros([ 400, 400, 3])
transforms.Numpy2Tensor()(img)

In [ ]:
bboxes = np.random.rand(20, 4)

In [ ]:
from mmcv.image.transforms import geometry

In [ ]:
geometry.impad(img, (411, 411), pad_val=2).shape

In [ ]:
pad = np.empty((411,411))